<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/LSTM-inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
scaler = StandardScaler()
encoder = LabelEncoder()

In [6]:
train = pd.read_csv('/content/drive/MyDrive/train-intervehicle.csv', chunksize=500000)
test = pd.read_csv('/content/drive/MyDrive/test-intervehicle.csv')

In [7]:
model = Sequential()
model.add(LSTM(100, input_shape=(len(test.columns)-1, 1), activation='tanh', recurrent_activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [8]:
for chunk in train:
  train_y = chunk.pop('Label')
  train_x = chunk
  train_x = scaler.fit_transform(train_x)
  train_y = encoder.fit_transform(train_y)
  model.fit(train_x, train_y, batch_size=32)

6161/6161 [==============================] - 31s 5ms/step - loss: 3.6896e-04 - accuracy: 0.9999


In [9]:
test_y = test.pop('Label')
test_x = test

In [10]:
test_x = scaler.transform(test_x)
test_y = encoder.transform(test_y)

In [11]:
pred = model.predict(test_x, verbose=1)
pred_y = np.round(pred).astype(int)

79666/79666 [==============================] - 204s 3ms/step


In [12]:
confusion_matrix(test_y, pred_y)

array([[1255574,     167],
       [      5, 1293542]])

In [13]:
report = classification_report(test_y, pred_y, digits=5)
print(report)

              precision    recall  f1-score   support

           0    1.00000   0.99987   0.99993   1255741
           1    0.99987   1.00000   0.99993   1293547

    accuracy                        0.99993   2549288
   macro avg    0.99993   0.99993   0.99993   2549288
weighted avg    0.99993   0.99993   0.99993   2549288

